
# 🧠 PJ Jewelry Image Sorter

Upload your images to your Google Drive under `/MyDrive/ImageFolder`.

Then run the cells below and click **Start Classification** to sort them into folders like:

- `ring`
- `bracelet`
- `watch`
- `engagement ring`
- `custom piece`

All sorted images will go inside `/MyDrive/ImageFolder/sorted-jewelry`.


In [ ]:
!git clone https://github.com/papasjorg/pj-image-sorter.git
import sys
sys.path.append('/content/pj-image-sorter')

In [ ]:
# PJ Jewelry Image Sorter 🧠💎

#Upload your images to your Google Drive under `/MyDrive/ImageFolder`.  
#Then run the cells below and click **Start Classification** to sort them into folders like:

#- `ring`
#- `bracelet`
#- `watch`
#- `engagement ring`
#- `custom piece`


#All sorted images will go inside `/MyDrive/ImageFolder/sorted-jewelry`.


### Mount Google Drive

from IPython.display import clear_output
import time
import os, sys
import ipywidgets as widgets
from IPython.display import display
import warnings

warnings.filterwarnings('ignore')

def is_running_in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

def create_mount_google_drive_button():
    mount_button = widgets.Button(
        description='Mount Google Drive',
        disabled=False,
        button_style='success',
        tooltip='Click to mount Google Drive',
        icon='upload'
    )
    def on_mount_drive(b):
        from google.colab import drive
        GDRIVE_PATH = "/content/drive/"
        drive.mount(GDRIVE_PATH)
        print(f"✅ Google Drive mounted at {GDRIVE_PATH}")
    mount_button.on_click(on_mount_drive)
    return mount_button

if is_running_in_colab():
    display(create_mount_google_drive_button())

# --- Main Tool Setup ---
def setup_notebook(gh_name, repo_name):
    if is_running_in_colab():
        # Check for GPU
        if (os.environ.get("COLAB_GPU")==''):
            print("\x1b[33m It looks like there isn't a GPU available, which could make the tool run very slowly. Try selecting a runtime with a GPU (Runtime > Change runtime type). \x1b[0m")

        # Clone Repo
        repo_url = f"https://github.com/{gh_name}/{repo_name}.git"

        if not os.path.exists(repo_name):
            clone_repository(repo_url, repo_name)

        repo_path = os.path.abspath(repo_name)

        if repo_path not in sys.path:
            sys.path.append(repo_path)

setup_notebook("papasjorg", "pj-image-sorter")

# Load helper functions
from utils.f_manager import setup_directories, list_images
from utils.model import classify_images

labels_input = widgets.Text(
    value="ring, engagement ring, bracelet, earring, necklace, pendant, watch, custom piece",
    placeholder='Enter comma-separated labels',
    description='Labels:',
    layout=widgets.Layout(width='50%'),
    disabled=False
)

source_input = widgets.Text(
    value="/content/drive/MyDrive/ImageFolder",
    placeholder='Enter source directory path',
    description='Source:',
    layout=widgets.Layout(width='50%'),
    disabled=False
)

destination_input = widgets.Text(
    value="sorted-jewelry",
    placeholder='Enter destination directory path',
    description='Destination:',
    layout=widgets.Layout(width='50%'),
    disabled=False
)

model_input = widgets.Combobox(
    value="openai/clip-vit-base-patch32",
    placeholder="Select or type model name",
    options=["openai/clip-vit-base-patch32"],
    description='Model:',
    ensure_option=True,
    disabled=False
)

operation_input = widgets.Dropdown(
    options=['copy', 'move'],
    value='copy',
    description='Operation:',
    disabled=False,
)

start_classification_button = widgets.Button(
    description='Start Classification',
    disabled=False,
    button_style='success',
    tooltip='Click to start image classification',
    icon='play'
)

def on_start_classification(b):
    clear_output()
    display(source_input, destination_input, model_input, labels_input, operation_input, start_classification_button)
    start_time = time.time()
    source = source_input.value
    destination = destination_input.value
    setup_directories(source, destination)
    labels = [label.strip() for label in labels_input.value.split(',')]
    if len(labels) < 2:
        print(f"⚠️ Please enter two or more labels.")
    else:
        print(f"Labels: {labels}")
        print(f"Source: {source}")
        print(f"Destination: {destination}")
        print("Starting classification...")
        images = list_images(source)
        classify_images(images, destination, model_input.value, labels, operation_input.value)
        elapsed_time = round(time.time() - start_time)
        print(f"✅ Done in {elapsed_time} seconds.")

start_classification_button.on_click(on_start_classification)

display(source_input, destination_input, model_input, labels_input, operation_input, start_classification_button)
